# American Community Survey (ACS)

In [1]:
from probplots import plot_dists_pps

from collections import OrderedDict
from imp import reload
import pdb
import random

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from IPython.display import SVG, display

from hud_geo_conversions import read_zips_to_fips
from map_maker import draw_county_data_svg
from utilities import (display_cb, is_outlier_instance, is_outlier_val,
                       boxcox_standardize, read_fips_codes)
from probplots import ProbPlots, plot_dists_pps



from ajpp import read_ajpp_geo, read_ajpp_pop
import jdata
from jewish_county_data import read_jdata_counties, clean_jdata_county
from acs import read_acs_israeli_ancestry
from religion_census import read_all_denoms, read_cb, read_judaic_denoms




pd.set_option('display.float_format', lambda x: '%.3f' % x)
from sklearn.decomposition import PCA

%matplotlib inline

SEED = 42
random.seed(SEED)

In [2]:
DATA_DIR    = '../Data/'
IMG_DIR = '../Images/'
COLORED_DIR = ''.join([IMG_DIR, 'colored.svg'])  # for playing around
DEM_DIR = ''.join([DATA_DIR, 'Demography/'])
SCL_DIR = ''.join([DATA_DIR, 'Schools/'])


zips_to_fips = read_zips_to_fips()  # used for multiple datasets later
fips_codes = read_fips_codes(''.join([DATA_DIR, 'Census-2010-County-FIPS.txt']))



B01003_FP = ''.join([DEM_DIR, 'ACS_15_5YR_B01003.csv'])
B04004_FP = ''.join([DEM_DIR, 'ACS_15_5YR_B04004.csv'])
B04005_FP = ''.join([DEM_DIR, 'ACS_15_5YR_B04005.csv'])

REL_DIR  = ''.join([DATA_DIR, 'Religion/'])
RELCEN_CB_FP  = ''.join([REL_DIR, 'US-Religion-Census-2010-County-File-CB.TXT'])
RELCEN_FP = ''.join([REL_DIR, 'US-Religion-Census-2010-County-File.DTA'])

JDATA_FP = ''.join([SCL_DIR, 'jdata_directory.json'])

AJPP_FP      = ''.join([DEM_DIR, 'AJPP_County2015.xlsx'])
AJPP_GEO_FP  = ''.join([DEM_DIR, 'AJPP_CountyGroupDefinitions2015.xlsx'])

In [3]:
acs = read_acs_israeli_ancestry(B01003_FP, B04004_FP, B04005_FP)
acs.head()

,County,State,Total_Pop,Only_Israeli_No,Only_Israeli_Pc,Part_Israeli_No,Part_Israeli_Pc,Tot_Israeli_No,Tot_Israeli_Pc
FIPS,,,,,,,,,
01001,Autauga County,AL,55221,0,0.000,0,0.000,0,0.000
01003,Baldwin County,AL,195121,0,0.000,0,0.000,0,0.000
01005,Barbour County,AL,26932,0,0.000,0,0.000,0,0.000
01007,Bibb County,AL,22604,0,0.000,7,0.031,7,0.031
01009,Blount County,AL,57710,0,0.000,0,0.000,0,0.000


In [4]:
acs.describe()

,Total_Pop,Only_Israeli_No,Only_Israeli_Pc,Part_Israeli_No,Part_Israeli_Pc,Tot_Israeli_No,Tot_Israeli_Pc
count,3220.000,3220.000,3220.000,3220.000,3220.000,3220.000,3220.000
mean,99409.346,27.646,0.006,15.798,0.005,43.444,0.011
std,319305.454,284.581,0.025,142.697,0.030,423.985,0.044
min,85.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,11218.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,26035.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,66430.500,0.000,0.000,0.000,0.000,0.000,0.000
max,10038388.000,11485.000,0.493,5719.000,0.697,17204.000,0.947


In [5]:
# frac of counties with more part than only
diff = acs.Only_Israeli_No - acs.Part_Israeli_No
sum(diff<0)/len(diff)

0.10186335403726708